# **CRDM Beta Simulation (CCB Project): Version 2**

#### Softmax Choice Model *only*
- Alpha [0.3, 1.2, 0.05] vs alpha constant [0.6]
    - While I realize the possible Alpha bounds swere [0.13, 4.68], iterating through this range with any fidelity would result in an unreasonable number of simulated Ss
    - So I narrowed the alpha range based on my experience with the value at which alpha began to fall apart with ADO
- Beta [-4.16, 4.16, 8.34/500], obtaining 500 steps (*simulated subjects*)
- Gamma constant [1.5]

##### **Version 2 Improvements** Restructures data into numpy arrays to improve computational efficiency (hrs --> secs)

In [ ]:
"""
===================
Mandy Renfro (2024)
===================
"""

from glob import glob
import numpy as np
np.seterr(all = "ignore")
import os
import os.path, sys
import pandas as pd
import pylab as plt
import seaborn as sns
sns.set_theme(style = "white", palette = "muted")
from scipy.stats import linregress, shapiro, spearmanr
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

base_proj_dir = "Z:/data/CCB" ## base project directory
save_dir = os.path.join(base_proj_dir, "derivatives/simulations/crdm/simulation2") ## save directory for CRDM simulations
if not os.path.exists(save_dir): ## new Ss
    os.makedirs(save_dir) ## make new Ss save directory

def pd_extend(df_dest, df_extending):
    """ Concatenates all subjects dataframe for saving current Ss parameters
        INPUT:
        - df_dest: destination dataframe to which second dataframe will be appended
        - df_extending: dataframe to be added to the destination dataframe
        OUTPUT:
        - Concatenated dataframe containing info from currently iterated Ss
    """
    return pd.concat([df_dest, df_extending])

def SVs(alpha, beta, lottery_value, certain_value, ambiguity, probability):
    """ Calculate SV for lottery and safe options given current trial condition combo (Gilboa & Schmeidler, 1989)
        *Note: np.sign() and np.abs() allows this function to flexibly handle *both* gain and loss trials
        INPUT:
        - alpha: current Ss alpha
        - beta: current Ss beta
        - lottery_value: winning lottery amount for current condition combo
        - ambiguity: ambiguity level for current condition combo
        - probability: probability for current condition combo
        OUTPUT:
        - SV_lottery: subjective value for lottery option for current trial condition combination
        - SV_certain: subjective value for certain option fr current trial condition combination
    """
    SV_lottery = (probability - beta * ambiguity / 2) * lottery_value**alpha
    SV_certain = certain_value**alpha
    return SV_lottery, SV_certain

In [ ]:
n_trials_per_subgroup = 2 ## number of trials per probability/lottery combo

lottery_range = [8, 25, 40, 50] ## possible winning lottery values
certain_value = 5 ## value of safe option
## setting up variable arrays for parallel computations
ambiguities_extend   = np.array([0.24,  0.5,  0.74,     0,    0,    0,    0,    0]) ## ambiguities for all amb/prob pairing
probabilities_extend = np.array([ 0.5,  0.5,   0.5,  0.13, 0.25, 0.38, 0.50, 0.75]) ## probabilties for all amb/prob pairing
lotteries = np.array([])
ambiguities = np.array([])
probabilities = np.array([])
extend_len = len(ambiguities_extend)
for i, lott in enumerate(lottery_range): ## for each of the four possible lottery values
    lotteries = np.hstack((lotteries, np.ones(extend_len) * lott)) ## adds current lott value in quantity of ambiguities_extend (8)
    ambiguities = np.hstack((ambiguities, ambiguities_extend)) ## adds ambiguity_extend as many times as lottery_range is long (4)
    probabilities = np.hstack((probabilities, probabilities_extend)) ## adds probabilities_extend as many times as lottery_range is long (4)
total_trials = n_trials_per_subgroup * len(lotteries) ## total number of trials per pain condition
print(total_trials)

### Determining Alpha and Beta bounds

In [ ]:
possible_alphas = []
possible_betas = []
probabilities_b = [0.13, 0.25, 0.38, 0.50, 0.75]
ambiguities_b = [0.24, 0.5, 0.74]

for this_prob in probabilities_b: ## for each probability level
    for this_val in lottery_range: ## and each lottery value level
        possible_alphas.append(np.log(this_prob)/np.log(certain_value/this_val)) ## append bound for current trial's variable level combo
possible_alphas = np.array(possible_alphas) ## convert list to a numpy array
print("ALPHA Bounds: [", np.amin(possible_alphas), ",", np.amax(possible_alphas), "]") ## print to output the lowest and highest beta bound

for this_amb in ambiguities_b: ## for each ambiguity level
    possible_betas.append(-1/this_amb) ## append lower bound for current beta
    possible_betas.append(1/this_amb) ## append upper bound for current beta
possible_betas = np.array(possible_betas) ## convert list to a numpy array
print("BETA Bounds: [", np.amin(possible_betas), ",", np.amax(possible_betas), "]") ## print to output the lowest and highest beta bound

## **Softmax Choice Model**

In [ ]:
def binomial_likelihood(alpha, beta, x,n,  lottery_value, certain_value, ambiguity, probability):
    """ Calculates binomial LL for each parameter combination.
        *See above notes for why we can use our own math rather than rely on bernoulli.logpmf()
        *Deals with positive infinity values by assigning a value of 0. 
        Then MLE functions, parameter pairs with a 0 LL score are removed.
        INPUT:
        - alpha: current Ss risk parameter [high values indicate risk seeking]
        - beta: current Ss ambiguity parameter [high values indicate ambiguity avoidance]
        - gamma: current Ss choice stochasticity parameter [high values indicate more noise]
        - y: summation of lottery choices in condition subgroup
        - n: number of trials per condition combo
        - lottery_value: current condition combo winning lottery amount 
        - certain_value: current condition combo certain amount
        - ambiguity: current condition combo ambiguity level
        - probability: current condition combo probability level
        OUTPUT:
        - log_likelihood: relative score representing probabilty data resulted from current parameter combo
    """
    p = probability_of_lottery_choice(alpha, beta, lottery_value, certain_value, ambiguity, probability)
    log_likelihood = (np.log(p[0]) * x + np.log(1 - p[0]) * (n - x))
    log_likelihood[log_likelihood == np.inf] = 0
    return log_likelihood

def make_data_and_recover(p_alpha, p_beta, pid, df):
    """ Executes parameter estimation and saves results to df_participants dataframe.
        INPUT:
        - p_alpha: current simulation Ss alpha value
        - p_beta: current simulation Ss beta value
        - pid: current simulation Ss subject ID
        - df: Ss dataframe with all previous Ss information (adds by extending current Ss df)
        OUPUT:
        - *res: recovered alpha, recovered beta
        - df: extended df_participants dataframe
    """
    pairs = []
    for i, amb in enumerate(ambiguities):
        subset_trials = np.random.random(size = n_trials_per_subgroup) ## simulated randomness of choice
        p_true = probability_of_lottery_choice(p_alpha, p_beta, lotteries[i], certain_value, amb, probabilities[i])
        idx = np.where(subset_trials <= p_true[0])
        sv_lott = p_true[1]
        sv_safe = p_true[2]
        subset_trials[:] = 0
        subset_trials[idx] = 1 ## error prone, real trials
        #subset_trials[:int(np.round(n_trials_per_subgroup*p_true))] = 1 ## perfect circumstances, no noise
        data = [pid, p_alpha, p_beta, None, None, lotteries[i], certain_value, amb, probabilities[i], sv_lott, sv_safe]
        data = [[a] * n_trials_per_subgroup for a in data]
        data.append(subset_trials)
        data = np.array(data).T
        df = pd_extend(df, pd.DataFrame(data, columns=df.columns))
        lottery_chosen_x = np.sum(subset_trials)
        pairs.append((lottery_chosen_x, n_trials_per_subgroup))
    res = MLE_alpha(np.array(pairs), lotteries, certain_value, ambiguities, probabilities)
    res = MLE_beta(np.array(pairs), res, lotteries, certain_value, ambiguities, probabilities)
    data = [pid, p_alpha, p_beta, res[0], res[1], None, None, None, None, None, None, None]
    data = np.array([[a] for a in data]).T
    df = pd_extend(df, pd.DataFrame(data, columns=df.columns))
    return *res, df

def MLE_alpha(pairs, lotteries, certain_value, ambiguities, probabilities):
    """ Grid search of alpha and gamma parameter space to determine which point produces the maximum log likelihood score. 
        *Risk trials only*
        *Notes: Conceptualize parameter and probability space as two parallel multidimensional spaces. 
                Using a "grid search" method, we carefully iterate through a 2D parameter space (alpha/gamma) 
                to determine which point best explains the data, quantified as the maximum likelihood score.
        INPUT:
        ** Numpy parallel arrays **
        - pairs: Ss choices for all condition subgroups
        - lotteries: winning lottery amounts for conditon subgroups
        - certain_values: certain amounts for condition subgroups
        - ambiguities: ambiguity level for condition subgroups
        - probabilities: probability level for condition subgroups
        OUTPUT:
        - Tuple containing best fit parameters (alpha and gamma)
    """
    best_fit = None
    max_likelihood = None
    x,n = pairs[:, 0], pairs[:, 1]
    idx = np.where(ambiguities == 0)
    for alpha in np.round(np.arange(0.3, 2, 0.01), 2): ## grid search - part 1 (fit alpha first)
        ## sums log likelihoods of all condition subgroups (beta held constant at 0, as is ambiguity level)
        likelihood = np.nansum(binomial_likelihood(alpha, 0, x[idx], n[idx], lotteries[idx], 
                                                    certain_value, ambiguities[idx], probabilities[idx]))
        if max_likelihood is None or likelihood > max_likelihood:
            max_likelihood = likelihood
            best_fit = alpha
    return best_fit

def MLE_beta(pairs, alpha, lotteries, certain_value, ambiguities, probabilities):
    """ Grid search of beta parameter space to determine which point produces maximum log likelihood score. 
        *Ambiguity trials only*
        *Notes: Conceptualize parameter and probability space as two parallel multidimensional spaces. 
        Using a "grid search" method previously to determine the best fit of alpha and gamma parameters,
        we can use those values to now guide our search through 1D beta space.
        INPUT:
        ** Numpy parallel arrays **
        - pairs: Ss choices for all condition subgroups
        - lotteries: winning lottery amounts for conditon subgroups
        - certain_values: certain amounts for condition subgroups
        - ambiguities: ambiguity level for condition subgroups
        - probabilities: probability level for condition subgroups
        OUTPUT:
        - Tuple containing best fit parameters (alpha, beta, and gamma)
    """
    best_fit = None
    max_likelihood = None
    x,n = pairs[:, 0], pairs[:, 1]
    idx = np.where(ambiguities != 0)
    for beta in np.round(np.arange(-1.3, 1.31, 0.01), 2): ## grid search - part 2 (fit beta second)
        ## sums log likelihoods of all condition subgroups (trials where ambiguity isn't 0)
        likelihood = np.nansum(binomial_likelihood(alpha, beta, x[idx], n[idx], lotteries[idx], 
                                                    certain_value, ambiguities[idx], probabilities[idx]))
        if max_likelihood is None or likelihood > max_likelihood:
            max_likelihood = likelihood
            best_fit = (alpha, beta)
    return best_fit

def probability_of_lottery_choice(alpha, beta, lottery_value, certain_value, ambiguity, probability, gamma = 1.5): 
    """ Determines probability of selecting lottery using the Softmax probabilitic function
        INPUT:
        - alpha: current Ss risk parameter [high values indicate risk seeking]
        - beta: current Ss ambiguity parameter [high values indicate ambiguity avoidance]
        - gamma: current Ss choice stochasticity parameter [high values indicate more noise]
        - lottery_value: current condition combo winning lottery amount 
        - certain_value: current condition combo certain amount
        - ambiguity: current condition combo ambiguity level
        - probability: current condition combo probability level
        OUTPUT:
        - Ss probability of choosing lottery for trials with the current condition combination 
    """
    SV_lottery, SV_certain = SVs(alpha, beta, lottery_value, certain_value, ambiguity, probability)
    return 1 / (1 + np.exp(-gamma * (SV_lottery - SV_certain))), SV_lottery, SV_certain

### **Softmax #1**
- Simulating across beta
- Alpha and gamma held constant (0.6, 1.5)

In [ ]:
alphas = np.array([0.6])
#betas = np.round(np.arange(-1.3, 1.31, 2.6/498), 2)
betas = np.round(np.arange(-4.17, 4.17, 8.34/500), 2) ## 500 steps
print("Number of Simulations:", len(alphas)*len(betas))
df = pd.DataFrame() ## new dataframe for correlation plot

## dataframe containing all Ss values to be saved to CSV
df_participants = pd.DataFrame(columns=["PID", 
                                        "Simulation Alpha", "Simulation Beta", 
                                        "Recovered Alpha", "Recovered Beta", 
                                        "Lottery Value", "Safe Value", 
                                        "Ambiguity", "Probability", 
                                        "SV Lottery", "SV Safe", 
                                        "Choice"])
recovered_betas = []
pid = 1
for p_alpha in alphas:
    for p_beta in betas:
        alpha_r, beta_r, df_participants = make_data_and_recover(p_alpha, p_beta, pid, df_participants)
        recovered_betas.append(beta_r)
        print(pid, end = "\r")
        pid += 1      
filename2 = os.path.join(save_dir, "simulation2SM_cAvBcG-2trials.csv") ## path/filename to save file
df_participants.to_csv(filename2) ## save DF to csv
beta_x = betas.tolist() * alphas.shape[0] ## reshape beta list
fit_line = linregress(beta_x, recovered_betas) ## create regression line
fit_stat = spearmanr(beta_x, recovered_betas) ## non-normal distribution
line_y = np.array(beta_x) * fit_line.slope + fit_line.intercept
residuals = np.array(recovered_betas) - line_y ## get residual for each beta point
print("Shapiro-Wilk Test of Normality:", shapiro(np.abs(residuals))[:]) ## print outcome of normality test
print("Spearmans Rho^2:", fit_stat[:]) ## print Spearman's Rho^2 (non-parameter correlation)
print("Residual SD:", np.std(residuals)) ## print SD of the residuals
df["Simulated β"] = beta_x ## assign x axis values
df["Fit β"] = recovered_betas ## assign y axis values
fig, ax = plt.subplots(figsize=(8, 8))
ax = sns.regplot(x = "Simulated β", y = "Fit β", data = df, color = "#26B64A") ## use seaborn regplot
sns.despine(top = True) ## remove upper and right plot blox line
ax.set_ylabel("Fit β", fontsize=16) ## set font size for y axis
ax.set_xlabel("Simulated β", fontsize=16) ## set font size for x axis
## set title for figure to include important stats
plt.suptitle("n={5} (α=0.6) | Shapiro-Wilk={0:.2f}, p={1:.2f}** | Spearmans Rho^2={2:.2f}, p={3:.2f}** | Residual SD={4:.2f}" .format(shapiro(np.abs(residuals))[0], 
                                                                                                                                    shapiro(np.abs(residuals))[1],
                                                                                                                                    fit_stat[0]**2, fit_stat[1], 
                                                                                                                                    np.std(residuals),
                                                                                                                                    len(beta_x)), 
                                                                                                                                    fontsize=12)
fig_name = os.path.join(save_dir, "simulation2SM_cAvBcG-2trials.png") ## directory and filename for beta plot
print("Saving to: {}".format(fig_name)) ## indicate save directory
plt.savefig(fig_name) ## save plot

### **Softmax #2**
- Simulating across alpha and beta
- Gamma held constant (1.5)

In [ ]:
alphas = np.round(np.arange( 0.3,  1.2,     0.05), 2) ## 18 steps
betas = np.round(np.arange(-4.17, 4.17, 8.34/500), 2) ## 500 steps
print("Number of Simulations:", len(alphas)*len(betas))
df = pd.DataFrame()

df_participants = pd.DataFrame(columns=["PID", 
                                        "Simulation Alpha", "Simulation Beta", 
                                        "Recovered Alpha", "Recovered Beta", 
                                        "Lottery Value", "Safe Value", 
                                        "Ambiguity", "Probability", 
                                        "SV Lottery", "SV Safe", 
                                        "Choice"])
recovered_betas = []
pid = 1
for p_alpha in alphas:
    for p_beta in betas:
        alpha_r, beta_r, df_participants = make_data_and_recover(p_alpha, p_beta, pid, df_participants)
        recovered_betas.append(beta_r)
        print(pid, end = "\r")
        pid += 1      
filename = os.path.join(save_dir, "simulation2SM_vAvBcG-2trials.csv")
df_participants.to_csv(filename)
        
beta_x = betas.tolist() * alphas.shape[0]
fit_line = linregress(beta_x, recovered_betas)
fit_stat = spearmanr(beta_x, recovered_betas)
line_y = np.array(beta_x) * fit_line.slope + fit_line.intercept
residuals = np.array(recovered_betas) - line_y
print("Shapiro-Wilk Test of Normality:", shapiro(np.abs(residuals))[:])
print("Spearmans Rho^2:", fit_stat[:])
print("Residual SD:", np.std(residuals))
df["Simulated β"] = beta_x
df["Fit β"] = recovered_betas
fig, ax = plt.subplots(figsize=(8, 8))
ax = sns.regplot(x = "Simulated β", y = "Fit β", data = df)
sns.despine(top = True)
ax.set_ylabel("Fit β", fontsize=16)
ax.set_xlabel("Simulated β", fontsize=16)
plt.suptitle("n={5} (α[0.3,1.2,0.05]) | Shapiro-Wilk={0:.2f}, p={1:.2f}** | Spearmans Rho^2={2:.2f}, p={3:.2f}** | Residual SD={4:.2f}" .format(shapiro(np.abs(residuals))[0], 
                                                                                                                                                shapiro(np.abs(residuals))[1], 
                                                                                                                                                fit_stat[0], fit_stat[1], 
                                                                                                                                                np.std(residuals),
                                                                                                                                                len(beta_x)), 
                                                                                                                                                fontsize=12)
fig_name = os.path.join(save_dir, "simulation2SM_vAvBcG-2trials.png")
print("Saving to: {}".format(fig_name))
plt.savefig(fig_name)